# Akka.Streams Graph Lifecycles
Just like how Akka.NET actors have discrete lifecycles, so do Akka.Streams graphs!

* **Running** – the stream is currently processing elements and has not run out of elements to process.
* **Completed** – the stream has terminated because:
    * The `Source<T>` ran out of elements or
    * One or more downstream stages signaled completion.
* **Failed** – the stream terminated because an error was thrown in one of the stages.

Let's consider some examples.

## Completed Streams
A stream completes usually when it's completed processing all possible inputs - or if one of the graph stages signals that it wants to terminate, i.e. a [`KillSwitch`](https://getakka.net/api/Akka.Streams.IKillSwitch.html).

In [ ]:
#r "nuget: Akka.Streams, 1.4.24"
#r "nuget: Akka.Streams.IAsyncEnumerable, 0.1.0"

using System.Linq;
using System.Collections.Immutable;
using Akka;
using Akka.Actor;
using Akka.Streams;
using Akka.Streams.Dsl;

ActorSystem actorSystem = ActorSystem.Create("StreamsExample");

IMaterializer materializer = actorSystem.Materializer();

// a source representing a range of integers
Source<int, NotUsed> source1 = Source.From(Enumerable.Range(1, 10));

// a source representing a single string value
Source<string, NotUsed> source2 = Source.Single("a");

// let's combine these two sources such that we create 10 int / string tuples
IAsyncEnumerable<(int i, string s)> merged1 = source1.Zip(source2).RunAsAsyncEnumerable(materializer);

await foreach(var (i, s) in merged1){
    Console.WriteLine($"{i}-->{s}");
}

Installed Packages Akka.Streams, 1.4.24 Akka.Streams.IAsyncEnumerable, 0.1.0

1-->a


This stream terminated with a single output, despite the fact that we had `source1` defined as 

```csharp
Source<int, NotUsed> source1 = Source.From(Enumerable.Range(1, 10));
```

Which is at least 10 elements! What gives?

Well in this case, it's because we combined `source1` with `source2`, which is defined as

```csharp
Source<string, NotUsed> source2 = Source.Single("a");
```

A single element! So due to the semantics of how the `Zip<T>` stage works - it will "complete" once at least one of its upstream `Source<T>`s completes, the entire stream terminates with a single output.

So how could we fix this in order to ensure that we had all 10 outputs from `source1` get rendered?

In [ ]:
// a source representing a single string value
// BUT this source will be repeated each time a downstream requests it.
Source<string, NotUsed> source3 = Source.Repeat("a");

// let's combine these two sources such that we create 10 int / string tuples
IAsyncEnumerable<(int i, string s)> merged1 = source1.Zip(source3).RunAsAsyncEnumerable(materializer);

await foreach(var (i, s) in merged1){
    Console.WriteLine($"{i}-->{s}");
}

1-->a
2-->a
3-->a
4-->a
5-->a
6-->a
7-->a
8-->a
9-->a
10-->a


Now we've had all 10 values from `source1` written out, all because we changed to a new `Source<T>` that had different semantics.

> **N.B.** This type of use case is exactly why it's a good idea to read "[Overview of Built-In Akka.Streams Stages and Their Semantics](https://getakka.net/articles/streams/builtinstages.html)" as it spells out exactly how these streams stages work with regard to lifetime, completion order, and so on.

## Infinite Streams
Many of the examples we've used so far have been for "finite" streams - where the number of possible inputs and outputs can be determined ahead of actually running the code.

I.e. when we create a `Source<int>` from a `List<int>`, we know how many items in the `List<int>` will need to be processed before we reach the end. That's an example of a finite stream.

An infinite stream, on the other hand, is unbounded - it has no discrete end, so it runs until it's told to terminate via some external or internal signal such as an `IKillSwitch`.

In [ ]:
// create a source that will be materialized into an IActorRef
Source<string, IActorRef> actorSource = Source.ActorRef<string>(1000, OverflowStrategy.DropTail);
var (preMaterializedRef, standAloneSrc) = actorSource.PreMaterialize(materializer);

// materialize the rest of the stream into an IAsyncEnumerable
IAsyncEnumerable<string> strResponses = standAloneSrc.Via(Flow.Create<string>().Select(x => x.ToLowerInvariant())).RunAsAsyncEnumerable(materializer);

// send some messages to our head actor to drive the stream
preMaterializedRef.Tell("HIT1");
preMaterializedRef.Tell("HIT2");
preMaterializedRef.Tell("HIT3");

await foreach(var str in strResponses){
    Console.WriteLine(str);
}